# Notebook: Calculate Agreement

This notebook is used to calculate the inter-rater agreement using Krippendorf's Alpha.
<br>**Contributors:** [Nils Hellwig](https://github.com/NilsHellwig/) | [Markus Bink](https://github.com/MarkusBink/)

## Packages

In [8]:
from statsmodels.stats import inter_rater as irr
import krippendorff as kd
import pandas as pd
import numpy as np
import glob
import os

## Parameters

In [9]:
ANNOTATED_DATASET_PATH = "../Datasets/annotated_dataset/"
LABEL_CODING = {'NEUTRAL': 3, 'NEGATIVE': 2, 'POSITIVE': 1, 'MIXED': 4}

## Code

### 1. Load Annotations

In [10]:
file_list = sorted(glob.glob(ANNOTATED_DATASET_PATH + "*.xlsx"))

['../Datasets/annotated_dataset/']

In [11]:
df_ann_1_session_1 = pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_1_1.xlsx")
df_ann_1_session_2 = pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_2_1.xlsx")
df_annotator_1 = pd.concat([df_ann_1_session_1, df_ann_1_session_2])
df_annotator_1["sentiment"] = df_annotator_1["sentiment"]
df_annotator_1.rename(columns={'sentiment': 'sentiment_1'}, inplace=True)
df_annotator_1 = df_annotator_1.reset_index(drop=True)

In [12]:
df_ann_2_session_1 = pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_1_2.xlsx")
df_ann_2_session_2 = pd.read_excel(ANNOTATED_DATASET_PATH + "tweets_session_2_2.xlsx")
df_annotator_2 = pd.concat([df_ann_2_session_1, df_ann_2_session_2])
df_annotator_2["sentiment"] = df_annotator_2["sentiment"]
df_annotator_2.rename(columns={'sentiment': 'sentiment_2'}, inplace=True)
df_annotator_2 = df_annotator_2.reset_index(drop=True)

In [13]:
df_all_annotations = pd.concat([df_annotator_1[['id', 'sentiment_1']], df_annotator_2['sentiment_2']], axis=1).reset_index(drop=True)

In [14]:
df_all_annotations['sentiment_1'] = df_all_annotations['sentiment_1'].map(LABEL_CODING)
df_all_annotations['sentiment_2'] = df_all_annotations['sentiment_2'].map(LABEL_CODING)

In [15]:
#df_all_annotations['sentiment_2'].unique()
df_all_annotations[df_all_annotations['sentiment_2'].isnull()]

,id,sentiment_1,sentiment_2


In [16]:
df_all_annotations

,id,sentiment_1,sentiment_2
0,1460589265759480064,2,4
1,1470141403045019904,3,2
2,1405950770466497024,3,3
3,1350126525220314880,3,3
4,1443988844500733952,2,2
...,...,...,...
1995,1400720644279504896,2,2
1996,1415980314871087104,3,3
1997,1419538832488374016,2,3
1998,1436026687548953088,3,3


In [17]:
equal_values = len(df_all_annotations[df_all_annotations['sentiment_1'] == df_all_annotations['sentiment_2']])
equal_values

1674

### 2. Calculate Krippendorff's Alpha

In [18]:
# Rows are the coders (annotators) # of coders
# Columns are the individual items (sentiment of tweet) # of tweets
value_counts = df_all_annotations.loc[:, df_all_annotations.columns != 'id']
value_counts = value_counts.to_numpy().transpose()
kd.alpha(reliability_data=value_counts, level_of_measurement="nominal")

0.7266937120270791

### 3. Calculate Fleiss' Kappa

In [19]:
agg = irr.aggregate_raters(df_all_annotations.loc[:, df_all_annotations.columns != 'id'])

In [20]:
irr.fleiss_kappa(agg[0], method='fleiss')

0.7266253683691716